In [1]:
# importing libraries 
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error

# 1. Python: Download a timeseries of daily deaths per country

In [2]:
#reading the data from the csv file
global_ds = pd.read_csv('C:/Users/Jewin/Desktop/Jovita-DataSciene/DATA/time_series_covid19_deaths_global.csv')

In [3]:
#displaying the raw data imported
global_ds

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,1,1,1,2,4,4,4,4,4,4
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2,4,5,5,6,8,10,10,11,15
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,17,17,19,21,25,26,29,31,35,44
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,1,1,1,1,3,3,3,6,8,12
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Turks and Caicos Islands,United Kingdom,21.694000,-71.797900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
252,NaN,MS Zaandam,0.000000,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253,NaN,Botswana,-22.328500,24.684900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
254,NaN,Burundi,-3.373100,29.918900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#deleting the columns not required for analysis
global_ds=global_ds.drop(columns=['Province/State','Lat','Long'])
#Renaming the column name
global_ds=global_ds.rename(columns={"Country/Region": "country"})

# 2. Python: Convert the table so that each country and each day is a separate row

In [5]:
#using the melt function to convert the columns to rows 
data=global_ds.melt(id_vars=["country"], 
        var_name="Date", 
        value_name="Death_Count")

In [6]:
#Checking the datatypes
data.dtypes

country        object
Date           object
Death_Count     int64
dtype: object

In [7]:
#converting the Datetype of Date column from object to datetime 
data['Date'] = pd.to_datetime(data['Date'])

In [8]:
#Clubing the different regions within the country to get country wise data
data = data.groupby(['country','Date'], as_index=False, sort=False)['Death_Count'].sum()

In [9]:
#displaying the final dataset 
data

,country,Date,Death_Count
0,Afghanistan,2020-01-22,0
1,Albania,2020-01-22,0
2,Algeria,2020-01-22,0
3,Andorra,2020-01-22,0
4,Angola,2020-01-22,0
...,...,...,...
12595,Burma,2020-03-31,1
12596,MS Zaandam,2020-03-31,0
12597,Botswana,2020-03-31,1
12598,Burundi,2020-03-31,0


# 3. Python: Provide code to upload the table from step 3 into an SQL table named deaths_total

In [10]:
try:
    sqliteConnection = sqlite3.connect('Covid19.db')
    c = sqliteConnection.cursor()
    c.execute('DROP TABLE IF EXISTS deaths_total')
    c.execute('CREATE TABLE deaths_total(country,Date,Death_Count)')
    data.to_sql('deaths_total', sqliteConnection, if_exists='replace', index = False)
    sqliteConnection.commit()
    c.close()
except sqlite3.Error as error:
    print("Error while executing sqlite script", error)
finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("sqlite connection is closed")

sqlite connection is closed


# 4. Python: From the data in step 2, calculate the daily change in deaths for each country

In [11]:
data['Daily_Deaths'] = ((data['Death_Count']-data.sort_values(by=['Date'], ascending=True)
                       .groupby(['country'])['Death_Count'].shift(1,fill_value=0)))

In [12]:
#displaying the first 10 rows displaying daily deaths in China
data[data['country']=='China'].head(10)

,country,Date,Death_Count,Daily_Deaths
32,China,2020-01-22,17,17
212,China,2020-01-23,18,1
392,China,2020-01-24,26,8
572,China,2020-01-25,42,16
752,China,2020-01-26,56,14
932,China,2020-01-27,82,26
1112,China,2020-01-28,131,49
1292,China,2020-01-29,133,2
1472,China,2020-01-30,171,38
1652,China,2020-01-31,213,42


In [13]:
#displaying the lasr 10 rows displaying daily deaths in Australia
data[data['country']=='Australia'].tail(10)

,country,Date,Death_Count,Daily_Deaths
10808,Australia,2020-03-22,7,0
10988,Australia,2020-03-23,7,0
11168,Australia,2020-03-24,8,1
11348,Australia,2020-03-25,8,0
11528,Australia,2020-03-26,13,5
11708,Australia,2020-03-27,13,0
11888,Australia,2020-03-28,14,1
12068,Australia,2020-03-29,16,2
12248,Australia,2020-03-30,17,1
12428,Australia,2020-03-31,18,1


# 5. Python: Provide code to upload the table from step 4 into an SQL table named deaths_change_python

In [14]:
try:
    sqliteConnection = sqlite3.connect('Covid19.db')
    c = sqliteConnection.cursor()
    c.execute('DROP TABLE IF EXISTS deaths_change_python')
    c.execute('CREATE TABLE deaths_change_python(country,Date,Death_Count,Daily_Deaths)')
    data.to_sql('deaths_change_python', sqliteConnection, if_exists='replace', index = False)
    sqliteConnection.commit()
    c.close()
except sqlite3.Error as error:
    print("Error while executing sqlite script", error)
finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("sqlite connection is closed")

sqlite connection is closed


# 6. SQL: Provide SQL code to calculate the daily change for each country using only the data from deaths_total and save it into an SQL table named deaths_change_sql

In [15]:
try:
    sqliteConnection = sqlite3.connect('Covid19.db')
    c = sqliteConnection.cursor()
    c.execute('DROP TABLE IF EXISTS deaths_change_sql')
    c.execute("CREATE TABLE deaths_change_sql AS SELECT country,Date,Death_Count,Death_Count-lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date) as Daily_Deaths from deaths_total")
    sqliteConnection.commit()
    c.close()
except sqlite3.Error as error:
    print("Error while executing sqlite script", error)
finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("sqlite connection is closed")

sqlite connection is closed
